## Dataset and Dataloader in Pytorch

### Dataset Class
The Dataset class is essentially a blueprint. When you create a custom dataset, you decide how dataset is loaded and returned.

It defines:
- __init__() which tells how the data should be loaded.
- __len__() which returns the total number of samples.
- __getitem__(index) which returns the data (and label) at the given index.

### DataLoader Class
The DataLoader class wraps a dataset and handles batching, shuffling and parallel loading for you.

DataLoader control flow:
- At the start of each epoch, the DataLoader (if shuffle=True) shuffles indices(using a sampler).
- It divides the index into chunks of batch_size.
- For each index in the chunk, data samples are fetched from the Dataset object.
- The samples are then collected and combined into a batch (using collate_fn).
- The batch is returned to the main training loop.

*We can also use custom sampler/shuffle specially when the data is imbalanced and not even use shuffling when the data is sequential.*

Important Parameters in DataLoader:
- **dataset(mandatory:)** The dataset from which the dataloader will pull the data.
- **batch_size:** Number of samples per batch (default=1)
- **shuffle:** Shuffles the dataset indices each eppoch.
- **num_workers:** The number of worker processes used to load data in parallel.
- **pin_memory:** Improves speed using GPU.
- **drop_last:** Drop the last remaining rows which is not of batch size.
- **collate_fn:** Processes a list of samples into a batch.
- **sampler:** For custom sampling.

In [2]:
from sklearn.datasets import make_classification
import torch

In [4]:
# Step-1: Create a synthetic classification dataset using sklearn
X, y = make_classification(
    n_samples=10,
    n_features=2,
    n_informative=2,
    n_redundant=0,
    n_classes=2,
    random_state=42
)

In [7]:
X, y

(array([[ 1.06833894, -0.97007347],
        [-1.14021544, -0.83879234],
        [-2.8953973 ,  1.97686236],
        [-0.72063436, -0.96059253],
        [-1.96287438, -0.99225135],
        [-0.9382051 , -0.54304815],
        [ 1.72725924, -1.18582677],
        [ 1.77736657,  1.51157598],
        [ 1.89969252,  0.83444483],
        [-0.58723065, -1.97171753]]),
 array([1, 0, 0, 0, 0, 1, 1, 1, 1, 0]))

In [8]:
X.shape

(10, 2)

In [10]:
# convert the data into pytorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

In [12]:
from torch.utils.data import Dataset, DataLoader

In [13]:
class CustomDataset(Dataset):

    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, index):
        return self.features[index], self.labels[index]
        

In [15]:
dataset = CustomDataset(X, y)

In [16]:
len(dataset)

10

In [17]:
dataset[0]

(tensor([ 1.0683, -0.9701]), tensor(1))

In [19]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [21]:
for batch_features, batch_labels in dataloader:
    print(batch_features)
    print(batch_labels)
    print("-"*50)

tensor([[ 1.7273, -1.1858],
        [ 1.0683, -0.9701]])
tensor([1, 1])
--------------------------------------------------
tensor([[-0.7206, -0.9606],
        [ 1.7774,  1.5116]])
tensor([0, 1])
--------------------------------------------------
tensor([[-0.5872, -1.9717],
        [-2.8954,  1.9769]])
tensor([0, 0])
--------------------------------------------------
tensor([[-0.9382, -0.5430],
        [-1.1402, -0.8388]])
tensor([1, 0])
--------------------------------------------------
tensor([[-1.9629, -0.9923],
        [ 1.8997,  0.8344]])
tensor([0, 1])
--------------------------------------------------


## Training Pipeline using Dataset and Dataloader

Earlier we were using batch gradient descent for the training, its inefficient because:
- Not memory efficient
- Slow convergence
  
Now we need to use Mini-Battch gradient descent by dividng the data into fixed number of batches.

In [22]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [23]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [24]:
df.shape

(569, 33)

In [25]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

In [26]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


### train test split

In [27]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

### scaling

In [28]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [29]:
X_train

array([[-0.56372349, -1.23042356, -0.61898028, ..., -1.49649753,
        -0.97835535, -1.24475887],
       [ 0.24691144,  0.09905778,  0.1527504 , ..., -0.89480528,
        -0.42702879, -1.21931893],
       [-1.58326397, -1.14382002, -1.58438365, ..., -1.78207776,
        -0.5113677 , -0.28493089],
       ...,
       [ 1.61540265,  0.54845992,  1.60740988, ...,  1.69515449,
         1.75797363, -0.24783097],
       [ 0.58685511, -1.04785394,  0.52487259, ..., -0.04114862,
        -0.22086702, -0.72800996],
       [-1.18840632,  0.45483448, -1.21353006, ..., -1.38439341,
        -0.89089278, -0.17204113]])

In [30]:
y_train

316    B
165    B
175    B
454    B
65     M
      ..
474    B
382    B
343    M
406    B
425    B
Name: diagnosis, Length: 455, dtype: object

### Label Encoding

In [31]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [32]:
y_train

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0,

### Numpy arrays to PyTorch tensors

In [33]:
X_train_tensor = torch.from_numpy(X_train.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train.astype(np.float32))
y_test_tensor = torch.from_numpy(y_test.astype(np.float32))

In [34]:
X_train_tensor.shape

torch.Size([455, 30])

In [35]:
y_train_tensor.shape

torch.Size([455])

In [36]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):

  def __init__(self, features, labels):

    self.features = features
    self.labels = labels

  def __len__(self):

    return len(self.features)

  def __getitem__(self, idx):

    return self.features[idx], self.labels[idx]

In [37]:
train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
test_dataset = CustomDataset(X_test_tensor, y_test_tensor)

In [38]:
train_dataset[10]

(tensor([ 0.3195,  2.6410,  0.4868,  0.1851,  0.5946,  2.0407,  2.2128,  1.2386,
          1.1255,  1.2278, -0.4952,  0.0071, -0.2208, -0.3667, -0.7971,  1.3698,
          1.5145,  0.7173,  0.1161,  0.9228,  0.2679,  2.7818,  0.6590,  0.0696,
          0.3953,  3.3330,  4.3679,  1.8509,  1.8267,  3.0233]),
 tensor(1.))

In [39]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

### Defining the model

In [40]:
import torch.nn as nn


class MySimpleNN(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):

    out = self.linear(features)
    out = self.sigmoid(out)

    return out

### Important Parameters

In [43]:
learning_rate = 0.1
epochs = 50

In [44]:
# create model
model = MySimpleNN(X_train_tensor.shape[1])

# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# define loss function
loss_function = nn.BCELoss()

### Training Pipeline

In [45]:

# define loop
for epoch in range(epochs):

  for batch_features, batch_labels in train_loader:

    # forward pass
    y_pred = model(batch_features)

    # loss calculate
    loss = loss_function(y_pred, batch_labels.view(-1,1))

    # clear gradients
    optimizer.zero_grad()

    # backward pass
    loss.backward()

    # parameters update
    optimizer.step()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 0.2964325547218323
Epoch: 2, Loss: 0.16277925670146942
Epoch: 3, Loss: 0.09498952329158783
Epoch: 4, Loss: 0.0476507842540741
Epoch: 5, Loss: 0.13122215867042542
Epoch: 6, Loss: 0.11229165643453598
Epoch: 7, Loss: 0.11242257803678513
Epoch: 8, Loss: 0.06875989586114883
Epoch: 9, Loss: 0.0654139518737793
Epoch: 10, Loss: 0.21441605687141418
Epoch: 11, Loss: 0.07949630171060562
Epoch: 12, Loss: 0.0810241624712944
Epoch: 13, Loss: 0.007344622630625963
Epoch: 14, Loss: 0.04108551889657974
Epoch: 15, Loss: 0.6393920183181763
Epoch: 16, Loss: 0.04253356531262398
Epoch: 17, Loss: 0.002910461975261569
Epoch: 18, Loss: 0.00836997851729393
Epoch: 19, Loss: 0.047728680074214935
Epoch: 20, Loss: 0.049239061772823334
Epoch: 21, Loss: 0.009693736210465431
Epoch: 22, Loss: 0.0013225231086835265
Epoch: 23, Loss: 0.03619762510061264
Epoch: 24, Loss: 0.0044717937707901
Epoch: 25, Loss: 0.091910220682621
Epoch: 26, Loss: 0.0063848295249044895
Epoch: 27, Loss: 0.4241867661476135
Epoch: 28,

### Evaluation

In [46]:
# Model evaluation using test_loader
model.eval()  # Set the model to evaluation mode
accuracy_list = []

with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        # Forward pass
        y_pred = model(batch_features)
        y_pred = (y_pred > 0.8).float()  # Convert probabilities to binary predictions

        # Calculate accuracy for the current batch
        batch_accuracy = (y_pred.view(-1) == batch_labels).float().mean().item()
        accuracy_list.append(batch_accuracy)

# Calculate overall accuracy
overall_accuracy = sum(accuracy_list) / len(accuracy_list)
print(f'Accuracy: {overall_accuracy:.4f}')


Accuracy: 0.9531
